In [106]:
import requests
from datetime import date
import pandas.io.sql as psql
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime, timezone
from time import sleep
import json

In [107]:
username =  'zillyonweb' 
password = 'pactodb2020' 
ipaddress = 'dbproself01.proselfit.com.br' 
port = '5432' 
dbname = 'PROSELF_PRD2' 
connection = f'postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'  

# parameters for connection to the database transient (datalake)
usr_datalake =  'postgres' 
pwd_datalake = 'X%p85qG!D6j[MoU%4[gEz,8BfKe^^0' 
host_datalake = '172.16.22.53' 
port_datalake = '5432' 
dbname_datalake = 'db_selfit_datalake' 
cnx_datalake = f'postgresql://{usr_datalake}:{pwd_datalake}@{host_datalake}:{port_datalake}/{dbname_datalake}'  

In [108]:
query = '''
        select
            pessoa.codigo as PersonCode 
            ,left(pessoa.nome, strpos(pessoa.nome, ' ') -1 ) as personname
            --,'RENATO MARKETING' as lastname
            ,right(pessoa.nome, length(pessoa.nome) - STRPOS(pessoa.nome, ' ')) as lastname
            ,plano.descricao as PlanName
            ,(select '+55' || translate(telefone.numero, '(,),-, ', '') from telefone where pessoa = cliente.codigo and tipotelefone = 'CE' limit 1 ) as mobile
            ,(select '+55' || translate(telefone.numero, '(,),-, ', '') from telefone where pessoa = cliente.codigo and tipotelefone = 'RE' limit 1 ) as telephone
            ,(select email from email where email.pessoa = cliente.pessoa limit 1) as email
            ,contrato.situacao
            ,cidade.nome as city
            ,estado.descricao as state
            ,to_char(contrato.vigenciade, 'yyyy-MM-dd') as planstartdate
            ,to_char(contrato.vigenciaateajustada, 'yyyy-MM-dd') as planexpirationdate
            ,empresa.nome as unitname
            ,pessoa.cfp as document
            ,pessoa.sexo as gender
            ,to_char(pessoa.datanasc , 'yyyy-MM-dd')  as birthdate
            ,cliente.matricula as registernumber
            ,endereco.cep as addresscep
            --,replace(translate(endereco.cep::text, '-',''),'.','') as addresscep
            ,endereco.bairro as addressneighborhood
            ,endereco.numero as addressnumber
            ,endereco.endereco as address
            ,endereco.complemento as addresscomplement
            ,cidp.nome as addresscity
            ,estp.descricao as addressstate
        from cliente
            left join pessoa on pessoa.codigo = cliente.pessoa
            left join empresa on cliente.empresa = empresa.codigo
            left join cidade on cidade.codigo = empresa.cidade
            left join estado on estado.codigo = empresa.estado
            left join cidade cidp on cidp.codigo = pessoa.cidade
            left join estado estp on estp.codigo = pessoa.estado
            left join contrato on contrato.pessoa = pessoa.codigo
            left join plano on plano.codigo = contrato.plano
            left join endereco on endereco.pessoa = pessoa.codigo
        --where cliente.codigomatricula = 1420556 -- joel
        --where cliente.pessoa = 1277684 --rafael
         where contrato.situacao is not NULL
        order by contrato.datamatricula desc, case contrato.Situacao when 'AT' then 1	
            when 'TR' then 2	
            when 'IN' then 3	
            when 'CA' then 4
            else 5
            end asc
        limit 1000
    '''

In [109]:
query_control = '''select "Email" ,
case when "*Situação do Cliente" in ('Ex-Clientes','Ex-Cliente','Ex-cliente') then 'IN' --inativo (IN)
when "*Situação do Cliente" in ('Lead') then NULL -- se encaixa em qualquer um. -> NULL
when "*Situação do Cliente" in ('Visitante', 'Teste') then 'IN' -- pode ser qualquer um.. (AT,IN)?
when "*Situação do Cliente" in ('Ativo', 'Cliente') then 'AT' --ativo AT
when "*Situação do Cliente" in ('Cancelado') then 'CA' --cancelado CA
when "*Situação do Cliente" in ('Trancado') then 'TR'	-- trancado TR
when "*Situação do Cliente" in ('CANCELAMENTO ESPONTÂNEO') then 'CA' -- cancelado CA
when "*Situação do Cliente" in ('CANCELAMENTO COMPULSÓRIO') then 'CA' --cancelado CA
when "*Situação do Cliente" in ('EM ABERTO > 2 MESES') then 'AT' -- ativo AT
--when "*Situação do Cliente" is null then null
end as "situacao_cliente"
from transient."3rd_active_campaign_enviados_20220615"''' 


In [110]:
# criando a base
df_all = psql.read_sql(query,connection) # a enviar

In [111]:
df_transient = psql.read_sql(query_control,cnx_datalake) # tabela control

In [112]:
df_transient.rename(columns={'Email':'email', 'situacao_cliente':'situacao'},inplace=True)

In [113]:
df_all.head()

,personcode,personname,lastname,planname,mobile,telephone,email,situacao,city,state,...,gender,birthdate,registernumber,addresscep,addressneighborhood,addressnumber,address,addresscomplement,addresscity,addressstate
0,903768,FADSON,SANTOS SOUZA,DAY USE,+5598998244211,None,fadson.souza@yahoo.com.br,AT,ARACAJU,SERGIPE,...,M,1986-09-13,883050,49.045-015,LUZIA,112,RUA PADRE NESTOR SAMPAIO,- ATÉ 99997/99998,ARACAJU,SERGIPE
1,1485577,MARCIO,VINICIUS ARAUJO DE BARROS,PLANO SELF PLUS SITE,+5591988633147,None,mvtecnologia1@yahoo.com.br,AT,MANAUS,AMAZONAS,...,m,1971-03-04,1459904,None,None,None,None,None,None,None
2,1451852,ARTHUR,GOMES LIMA FRANCA,DAY USE,+5511999590584,None,arthurgomes@gmail.com,AT,VITÓRIA DA CONQUISTA,BAHIA,...,M,1993-07-26,1426045,41.180-710,SABOEIRO,1361,BECO DA CORUJA,,SALVADOR,BAHIA
3,1413480,PABLO,REGIS DA SILVA REIS,PLANO SELF PLUS SITE,+5586988844509,None,aciolipablo0@gmail.com,AT,TERESINA,PIAUÍ,...,m,2000-04-20,1387916,None,None,None,None,None,None,None
4,1486726,BRUNA,PESSOA GUERRA,PLANO SELF PLUS SITE,+5591989189527,+5591989189527,guerrabp@hotmail.com,AT,RECIFE,PERNAMBUCO,...,f,1984-05-26,1461093,None,None,None,None,None,None,None


In [114]:
df_transient.head()

,email,situacao
0,leandro_silvaf@yahoo.com.br,IN
1,leandro_silva_o@outlook.com,IN
2,leandro_sk__@hotmail.com,None
3,leandro_sobral@hotmail.com,IN
4,leandro_souzassa@hotmail.com,CA


In [115]:
df_transient.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 520021 entries, 0 to 520020
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   email     520021 non-null  object
 1   situacao  377592 non-null  object
dtypes: object(2)
memory usage: 7.9+ MB


In [116]:
#df_merge = pd.merge(df_all, df_transient, on='email', how='outer')

In [137]:
df_merge = df_all[~df_all['email'].isin(df_transient['email'])]
#tem 722 linhas.

In [118]:
df_test = df_all[df_all['email'].isin(df_transient['email'])]


In [122]:
df_all = df_all.drop_duplicates(subset = 'email',keep='last')

In [123]:
df_all['situacao'].loc[df_all['email'] == 'danielvmatos@hotmail.com']

376    IN
Name: situacao, dtype: object

In [143]:
count = 0
for email in df_test['email']:
    #print(email)
    aux1 = df_transient['situacao'].loc[df_transient['email'] == email]
    aux2 = df_all['situacao'].loc[df_all['email'] == email]
    #print(aux1.item())
    #print(aux2.item())
    #print("aux1 = {}, aux2 = {}".format(aux1.item(),aux2.item()))
    
    if(aux1.item() != aux2.item()):
        print("true")
        count += 1
        print(count)
        #adiciona no df_merge os caras que mudaram.
        df_merge =df_merge.append(df_all.loc[df_all['email'] == email], ignore_index='True')
        
    else: 
        print("false")
        

true
1
true
2
true
3
true
4
true
5
true
6
true
7
true
8
false
false
true
9
true
10
true
11
true
12
false
true
13
true
14
false
true
15
true
16
true
17
false
true
18
true
19
false
false
true
20
false
true
21
false
true
22
false
true
23
true
24
true
25
true
26
true
27
true
28
false
true
29
true
30
true
31
true
32
false
false
false
true
33
true
34
true
35
false
true
36
true
37
true
38
false
true
39
true
40
true
41
true
42
true
43
false
true
44
true
45
true
46
false
false
false
true
47
false
false
true
48
false
true
49
false
true
50
true
51
true
52
true
53
false
true
54
false
false
false
false
false
false
false
false
false
false
false
true
55
false
true
56
false
false
false
true
57
true
58
false
false
true
59
true
60
true
61
false
false
false
false
true
62
true
63
false
false
false
false
true
64
true
65
false
true
66
false
false
false
false
false
false
true
67
true
68
false
true
69
false
false
true
70
false
true
71
false
true
72
true
73
true
74
true
75
false
false
false
false
true
76
false

In [144]:
#depois de mudar. acrescentou mais 136 linhas
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 859 entries, 0 to 858
Data columns (total 24 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   personcode           859 non-null    int64 
 1   personname           859 non-null    object
 2   lastname             859 non-null    object
 3   planname             859 non-null    object
 4   mobile               807 non-null    object
 5   telephone            236 non-null    object
 6   email                858 non-null    object
 7   situacao             859 non-null    object
 8   city                 859 non-null    object
 9   state                859 non-null    object
 10  planstartdate        859 non-null    object
 11  planexpirationdate   859 non-null    object
 12  unitname             859 non-null    object
 13  document             859 non-null    object
 14  gender               853 non-null    object
 15  birthdate            854 non-null    object
 16  register

In [ ]:
#df_all.info()
# 999 colunas de email.

In [ ]:
#df_transient.info()
#520021 linhas de email.

In [ ]:
#df_merge.info()

In [ ]:
#df_merge.head()

In [ ]:
#count duplicates
df_merge['email'].nunique()

8931

In [ ]:
df_merge.email.duplicated().sum()

62

In [ ]:
#df_transient.rename(columns={'Email':'email'},inplace=True)

In [ ]:
#df_transient.info()

In [ ]:
#df_merge = pd.merge(df_all, df_transient, on='email', how='outer')

In [ ]:
#df_merge.info()

In [ ]:
#df_merge.head()